# PyCUDA installation

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 18.3MB/s 
     |████████████████████████████████| 61kB 11.1MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536587 sha256=4e1546309de3b3ae6261a28e2db5520d86114a71ce07d5338d80ee2a1a82215c
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.1-py2.py3-none-any.whl size=59602 sha256=b89ef666dcb791761819a51e2e2bc6458c552bbeb8e28abdad04f6eaf686a7f8
  Stored in directory: /root/.cache/pip/wheels/6f/da/1b/946775a88291378182ed92c9800d6d0ebc2a554cb89829cc24
Successfully built pycuda pytools




---



# Version 4: using ```ElementwiseKernel```

The first part of the code is the usual one.

In [0]:
import numpy as np

import pycuda.driver as cuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray

########
# MAIN #
########

start = cuda.Event()
end   = cuda.Event()

N = 100000

h_a = np.random.randn(1, N)
h_b = np.random.randn(1, N)

h_a = h_a.astype(np.float32)
h_b = h_b.astype(np.float32)

d_a = gpuarray.to_gpu(h_a)
d_b = gpuarray.to_gpu(h_b)

In this example, ```d_c``` is explicitly defined. This is necessary for the use of the ```ElementwiseKernel``` module.

In [0]:
d_c = gpuarray.empty_like(d_a)

Load the ```ElementwiseKernel``` module.

In [0]:
from pycuda.elementwise import ElementwiseKernel

The ```ElementwiseKernel``` enables to define only the kernel instructions to be elementwise executed within the kernel. Here, to generalize Version #1, a general linear combination between ```d_a``` and ```d_b``` is considered. A reference to the elementwise kernel is defined in ```lin_comb```.


In [0]:
lin_comb = ElementwiseKernel(
        "float *d_c, float *d_a, float *d_b, float a, float b",
        "d_c[i] = a * d_a[i] + b * d_b[i]")

Invoke the ```lin_comb``` function.

In [5]:
start.record()
lin_comb(d_c, d_a, d_b, 2, 3)
end.record() 
end.synchronize()
secs = start.time_till(end) * 1e-3
print("Processing time = %fs" % (secs))

Processing time = 0.088942s


In [7]:
'''start.record()
lin_comb(d_c, d_a, d_b, 1, 6)
end.record() 
end.synchronize()
secs = start.time_till(end) * 1e-3
print("Processing time = %fs" % (secs))'''

Processing time = 0.000111s


The last part is as usual.

In [6]:
h_c = d_c.get()

if np.array_equal(h_c, 2 * h_a + 3 * h_b):
  print("Test passed!")
else :
  print("Error!")

cuda.Context.synchronize()

Test passed!
